In [1]:
# translation-encoder.txt
# translation-transformer.txt

In [4]:
import numpy as np 
import tensorflow as tf 
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [5]:
data_path = [
    ("Go.", "Va !"),
    ("Run!", "Cours !"),
    ("Run.", "Cours !"),
    ("Who?", "Qui ?"),
    ("Wow!", "Ça alors !"),
    ("Fire!", "Au feu !"),
    ("Help!", "À l'aide !"),
    ("Stop!", "Arrête-toi !"),
    ("Wait!", "Attends !"),
    ("Hello!", "Bonjour !"),
    ("I see.", "Je comprends.")
]

#### Data Preprocessing

In [8]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for input_text, target_text in data_path:
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)

    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [13]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [15]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

In [16]:
num_decoder_tokens, num_encoder_tokens

(33, 24)

In [22]:
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [21]:
max_encoder_seq_length

6

In [23]:
max_decoder_seq_length

15

In [26]:
# Create token mappings (char -> int)
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [29]:
# Create reverse token mappings (int -> char)
reverse_input_token_index = dict([(i,char) for i, char in enumerate(input_characters)])
reverse_target_token_index = dict([(i,char) for i, char in enumerate(target_characters)])

In [31]:
# Generate one-hot encoded data 
# Encoder Input -> (num_samples, max_len, unique_chars)
encoder_data_input = np.zeros((len(input_texts), 
                               max_encoder_seq_length,
                              num_encoder_tokens))

In [36]:
encoder_data_input.shape

(11, 6, 24)

In [46]:
# Decoder Input
decoder_data_input = np.zeros((len(input_texts), 
                               max_decoder_seq_length,
                              num_decoder_tokens))

In [47]:
decoder_data_input.shape

(11, 15, 33)

In [48]:
decoder_target_data = np.zeros((len(input_texts), 
                               max_decoder_seq_length,
                              num_decoder_tokens))

In [49]:
decoder_target_data.shape

(11, 15, 33)

In [50]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_data_input[i, t, input_token_index[char]] = 1
    encoder_data_input[i, t+1:, input_token_index[' ']] = 1 # padding

    for t, char in enumerate(target_text):
        decoder_data_input[i, t, target_token_index[char]] = 1
        if t > 0:
            decoder_data_input[i, t-1, target_token_index[char]] = 1

    decoder_data_input[i, t+1:, target_token_index[' ']] = 1 # padding
    decoder_target_data[i, t:, target_token_index[' ']] = 1 # padding

In [51]:
decoder_target_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0.

#### Build the model

In [58]:
# Create the encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

In [60]:
# Create the decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_state=True, return_sequences=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, 
                                     initial_state = [state_h, state_c])
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [61]:
# define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [68]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
             metrics=['accuracy']);

In [69]:
model.fit([encoder_data_input, decoder_data_input], decoder_target_data,
         epochs=100)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3636 - loss: 0.0786
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.3636 - loss: 0.0489
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3636 - loss: 0.1063
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.3636 - loss: 0.0492
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3636 - loss: 0.2319
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.3636 - loss: 0.1321
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.3636 - loss: 0.0848
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3636 - loss: 0.1308
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3636 - loss: 0.0705
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3636 - loss: 0.0959
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.3636 - loss: 0.0584
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.3636 - loss